# Tugas Teknik Optimasi Week 3
Airlangga Rasyad Fidiyanto <br/>
19/443562/TK/48758

In [1]:
import numpy as np

from scipy import optimize
from time import time

In [2]:
__author__ = "Airlangga Rasyad Fidiyanto"

In [3]:
myFunction = lambda x: x**3 - 2*x - np.exp(x)

lowerBound = -1
upperBound = 4

n = 10
dx = (upperBound-lowerBound) / n

errTarget = 0.01

In [4]:
minima = optimize.fmin(myFunction, 1)

Optimization terminated successfully.
         Current function value: -4.114712
         Iterations: 14
         Function evaluations: 28


In [5]:
minima = float(minima)

# Essential Functions

In [6]:
def generateFibonacci(arrayLength):
    fibonacciArray = [1, 1]
    
    for i in range(arrayLength - 2):
        result = fibonacciArray[-1]  + fibonacciArray[-2]
        fibonacciArray.append(result)
        
    return fibonacciArray

def firstDerivative(usrFunction, x):
    if abs(x) > 0.01:
        delta_x = abs(x) * 0.01
    else:
        delta_x = 0.0001
    return (usrFunction(x+delta_x) - usrFunction(x))/delta_x

def secondDerivative(usrFunction, x):
    if abs(x) > 0.01:
        delta_x = abs(x) * 0.01
    else:
        delta_x = 0.0001
    
    return (usrFunction(x+2*delta_x) - 2*usrFunction(x+delta_x) + usrFunction(x))/delta_x**2

def calculateError(y):
    return round(abs(y-minima)/minima * 100, 3)

def printResult(x_min, x_max, error, compTime, ctr, functEvalCtr, isInterval):   
    print(f"Iterations: {ctr}")
    print(f"Function Evaluations: {functEvalCtr}")
    
    if isInterval:
        print(f"Computed minima: At x = {x_min} - {x_max}")
        if error[0] > 100 or error[1] > 100:
            print(f"Error: \x1b[31m{error[0]}% - {error[1]}%% (WARNING: The method might not be accurate enough for this input!)\x1b[0m")
        else:
            print(f"Error: {error[0]}% - {error[1]}%")
            
    else:
        print(f"Computed minima: {y_min} at x = {x_min}")
        
        if error > 100: 
            print(f"Error: \x1b[31m{error}% (WARNING: The method might not be accurate enough for this input!)\x1b[0m")
        else: 
            print(f"Error: {error}%")
            
    print(f"Computation time: {compTime}s\n")

# Direct Search

In [7]:
def directSearch(a, b, usrFunction):
    start = time()
    
    iterationCounter = 0
    functionCallCounter = 0
    
    x1 = a
    x2 = x1 + dx
    x3 = x2 + dx

    while usrFunction(x1) >= usrFunction(x2) and usrFunction(x2) >= usrFunction(x3):       
        if x3 < b:
            x1 = x2
            x2 = x3
            x3 = x2 + dx
        else:
            return "There's no minima nor maxima"
        
        iterationCounter += 1
        functionCallCounter += 3
        
    end = time()
    
    compTime = end - start
    error = [calculateError(x3), calculateError(x1)]
    
    return [x1, x3, sorted(error), compTime, iterationCounter, functionCallCounter]

# Bounding Phase Search

In [8]:
def boundingPhase(a, b, delta, usrFunction):
    start = time()
    
    iterationCounter = 0
    functionCallCounter = 0
    
    x0 = a
    k = 0

    while True:
        iterationCounter += 1
        
        if usrFunction(x0-abs(delta)) >= usrFunction(x0) and usrFunction(x0) >= usrFunction(x0+abs(delta)):
            break
        elif usrFunction(x0-abs(delta)) <= usrFunction(x0) and usrFunction(x0) <= usrFunction(x0+abs(delta)):
            delta = -delta
            break
        else:
            x0 = x0 + delta

    while True:
        iterationCounter += 1
        functionCallCounter += 2
        
        x_new = x0 + delta*2**k
        if usrFunction(x_new) < usrFunction(x0):
            k += 1
            x_old = x0
            x0 = x_new
        else:
            end = time()
            
            compTime = end - start
            error = [calculateError(x_new), calculateError(x_old)]
            
            return [x_old, x_new, sorted(error), compTime, iterationCounter, functionCallCounter]
            break

# Interval Halving Search

In [9]:
#Set Initital Value
def intervalHalving(a, b, errTarget, usrFunction):
    start = time()
    
    iterationCounter = 0
    functionCallCounter = 0
    
    x_m = (a+b) / 2

    while (b-a) > errTarget:
        iterationCounter += 1
        functionCallCounter += 2
        
        x1 = a + ((b-a)/4)
        x2 = b - ((b-a)/4)

        if usrFunction(x1) < usrFunction(x_m):
            b  = x_m
            x_m = x1
        elif usrFunction(x2) < usrFunction(x_m):
            a  = x_m
            xm = x2
        else:
            a = x1
            b = x2
        end = time()
    
    compTime = end - start
    error = [calculateError(n), calculateError(a)]
    
    return [a, n, sorted(error), compTime, iterationCounter, functionCallCounter]

# Fibonacci Search

In [10]:
def fibonacciSearch(usrFunction, a, b, fibonacciLength):
    start = time()
    
    iterationCounter = 0
    functionCallCounter = 0
    
    fibonacciArray = generateFibonacci(fibonacciLength)

    while fibonacciArray[-1] + fibonacciArray[-2] != 2:
        fibonacciLastIndex = len(fibonacciArray) - 1

        c_index = -3
        d_index = -2


        c = a + (fibonacciArray[c_index]/fibonacciArray[fibonacciLastIndex] * (b - a))
        d = a + (fibonacciArray[d_index]/fibonacciArray[fibonacciLastIndex] * (b - a))

        if usrFunction(c) < usrFunction(d):
            b = d
            d = c
        else:
            d = c
            a = c

        fibonacciArray.pop()
        
        iterationCounter += 1
        functionCallCounter += 2
        
    end = time()
    
    compTime = end - start
    error = calculateError(c)
    
    return [c, usrFunction(c), error, compTime, iterationCounter, functionCallCounter]

# Golden Section Search

In [11]:
PHI = (1 + 5 ** 0.5) / 2

def goldenSection(usrFunction, a, b, errTarget):
    start = time()
    
    c = 0
    
    iterationCounter = 0
    functionCallCounter = 0
    
    while abs(c - minima) > errTarget:       
        c = b + (a - b) / PHI
        d = a + (b - a) / PHI

        if usrFunction(c) < usrFunction(d):
            b = d
            d = c
        else:
            d = c
            a = c
            
        iterationCounter += 1
        functionCallCounter += 2
            
    end = time()
    
    compTime = end - start
    error = calculateError(c)
    
    return [c, usrFunction(c), error, compTime, iterationCounter, functionCallCounter]

# Successive Quadratic

In [12]:
def successiveQuadratic(usrFunction, x1, delta, errTarget):
    start = time()
    
    iterationCounter = 0
    functionCallCounter = 0
    
    x_bar = 0
    x2 = x1 + delta
    
    if usrFunction(x1) > usrFunction(x2):
        x3 = x1 + (2 * delta)
    else:
        x3 = x1 - (2 * delta)
        
    threeBestVal = dict({x1:usrFunction(x1), x2:usrFunction(x2), x3:usrFunction(x3),})
    threeBestVal = {k: v for k, v in sorted(threeBestVal.items(), key=lambda item: item[1])}
    
    while abs(minima - x_bar) > errTarget:
        a1 = (threeBestVal[x2] - threeBestVal[x1]) / (x2 - x1)
        a2 = (1 / (x3 - x2)) * (((threeBestVal[x3] - threeBestVal[x1]) / (x3 - x1)) - a1)

        x_bar = ((x1 + x2) / 2) - (a1 / (2 * a2))
        threeBestVal[x_bar] = usrFunction(x_bar)

        threeBestVal = {k: v for k, v in sorted(threeBestVal.items(), key=lambda item: item[1])}
        
        threeBestValIndex = list(threeBestVal.keys())
        xBarLocation = threeBestValIndex.index(x_bar)
        
#         print(xBarLocation)
        
        if xBarLocation == 0:
            [x1, x2, x3] = threeBestValIndex[0:3]
        elif xBarLocation == len(threeBestVal) - 1:
            [x1, x2, x3] = threeBestValIndex[-3:]
        else:
            [x1, x2, x3] = threeBestValIndex[xBarLocation-1:xBarLocation+2]
            
        threeBestVal = dict({x1:usrFunction(x1), x2:usrFunction(x2), x3:usrFunction(x3),})
        
#         print(threeBestValIndex)
        
        iterationCounter += 1
        functionCallCounter += 1
    
    end = time()
    
    compTime = end - start
    error = error = calculateError(x_bar)
    
    return [x_bar, usrFunction(x_bar), error, compTime, iterationCounter, functionCallCounter]

# Newton-Rhapson Method

In [13]:
def newtonRhapson(x, errTarget, usrFunction):   
    start = time()
    
    iterationCounter = 0
    functionCallCounter = 0
    
    x_new = 0
    
    while abs(firstDerivative(usrFunction, x)) > errTarget:
        iterationCounter += 1
        functionCallCounter += 5
        
        x_new = x - (firstDerivative(usrFunction, x) / secondDerivative(usrFunction, x))
        x = x_new
        
        iterationCounter += 1
        functionCallCounter += 5 
        
    end = time()
    
    compTime = end - start
    error = calculateError(x_new)
    
    return [x_new, usrFunction(x_new), error, compTime, iterationCounter, functionCallCounter]

# Bisection Method

In [14]:
def bisection(usrFunction, a, b, errTarget):
    start = time()
    
    iterationCounter = 0
    functionCallCounter = 0
    
    x1 = a
    x2 = b
    z = (x1+x2) / 2
    
    while abs(firstDerivative(usrFunction, z)) > errTarget:
        z = (x1 + x2) / 2
        
        if firstDerivative(usrFunction, z) < 0:
            x1 = z
        elif firstDerivative(usrFunction, z) > 0:
            x2 = z
            
        iterationCounter += 1
        functionCallCounter += 2
            
    end = time()
    
    compTime = end - start
    error = calculateError(z)
    
    return [z, usrFunction(z), error, compTime, iterationCounter, functionCallCounter]

# Secant Method

In [15]:
def secant(usrFunction, a, b):
    start = time()

    iterationCounter = 0
    functionCallCounter = 0
    
    x1 = a
    x2 = b
    z = x2 - (firstDerivative(usrFunction, x2)*(x2-x1) / (firstDerivative(usrFunction, x2) - firstDerivative(usrFunction, x1)))
    
    while abs(firstDerivative(usrFunction, z)) > errTarget:
        z = x2 - (firstDerivative(usrFunction, x2)*(x2-x1) / (firstDerivative(usrFunction, x2) - firstDerivative(usrFunction, x1)))
        if firstDerivative(usrFunction, z) < 0:
            x1 = z
        elif firstDerivative(usrFunction, z) > 0:
            x2 = z
            
        iterationCounter += 1
        functionCallCounter += 6
    
    end = time()
    
    compTime = end - start
    error = calculateError(z)
    
    return [z, usrFunction(z), error, compTime, iterationCounter, functionCallCounter]

# Output Test

In [16]:
print("\x1b[35mDirect Search\x1b[0m")
[x_min, x_max, error, compTime, ctr, functEvalCtr] = directSearch(a = lowerBound, b = upperBound, usrFunction = myFunction)
printResult(x_min, x_max, error, compTime, ctr, functEvalCtr, True)

print("\x1b[35mBounding Phase Search\x1b[0m")
[x_min, x_max, error, compTime, ctr, functEvalCtr] = boundingPhase(a = lowerBound, b = upperBound, delta = dx, usrFunction = myFunction)
printResult(x_min, x_max, error, compTime, ctr, functEvalCtr, True)

print("\x1b[35mInterval Halving\x1b[0m")
[x_min, x_max, error, compTime, ctr, functEvalCtr] = intervalHalving(a = lowerBound, b = upperBound, errTarget = errTarget, usrFunction = myFunction)
printResult(x_min, x_max, error, compTime, ctr, functEvalCtr, True)

print("\x1b[35mFibonacci Search\x1b[0m")
[x_min, y_min, error, compTime, ctr, functEvalCtr] = fibonacciSearch(usrFunction = myFunction, a = lowerBound, b = upperBound, fibonacciLength = 10)
printResult(x_min, x_max, error, compTime, ctr, functEvalCtr, False)

print("\x1b[35mGolden Section Search\x1b[0m")
[x_min, y_min, error, compTime, ctr, functEvalCtr] = goldenSection(usrFunction = myFunction, a = lowerBound, b = upperBound, errTarget = errTarget)
printResult(x_min, x_max, error, compTime, ctr, functEvalCtr, False)

print("\x1b[35mSuccessive Quadratic Search\x1b[0m")
[x_min, y_min, error, compTime, ctr, functEvalCtr] = successiveQuadratic(usrFunction = myFunction, x1 = lowerBound, errTarget = errTarget, delta = 1)
printResult(x_min, x_max, error, compTime, ctr, functEvalCtr, False)

print("\x1b[35mNewton-Rhapson Method\x1b[0m")
[x_min, y_min, error, compTime, ctr, functEvalCtr] = newtonRhapson(x = lowerBound, usrFunction = myFunction, errTarget = errTarget)
printResult(x_min, x_max, error, compTime, ctr, functEvalCtr, False)

print("\x1b[35mBisection Method\x1b[0m")
[x_min, y_min, error, compTime, ctr, functEvalCtr] = bisection(a = lowerBound, usrFunction = myFunction, errTarget = errTarget, b = upperBound)
printResult(x_min, x_max, error, compTime, ctr, functEvalCtr, False)

print("\x1b[35mSecant Method\x1b[0m")
[x_min, y_min, error, compTime, ctr, functEvalCtr] = secant(a = lowerBound, usrFunction = myFunction, b = upperBound)
printResult(x_min, x_max, error, compTime, ctr, functEvalCtr, False)

Direct Search
Iterations: 4
Function Evaluations: 12
Computed minima: At x = 1.0 - 2.0
Error: 30.553% - 38.895%
Computation time: 8.320808410644531e-05s

Bounding Phase Search
Iterations: 5
Function Evaluations: 6
Computed minima: At x = 0.0 - 3.0
Error: 100.0% - 108.342%% (WARNING: The method might not be accurate enough for this input!)
Computation time: 3.314018249511719e-05s

Interval Halving
Iterations: 8
Function Evaluations: 16
Computed minima: At x = 1.421875 - 10
Error: 1.255% - 594.473%% (WARNING: The method might not be accurate enough for this input!)
Computation time: 5.1975250244140625e-05s

Fibonacci Search
Iterations: 8
Function Evaluations: 16
Computed minima: -4.114241900260672 at x = 1.4545454545454546
Error: 1.014%
Computation time: 3.409385681152344e-05s

Golden Section Search
Iterations: 11
Function Evaluations: 22
Computed minima: -4.11470289280064 at x = 1.44198596255573
Error: 0.142%
Computation time: 3.123283386230469e-05s

Successive Quadratic Search
Iteratio